In [ ]:
def permute(bits, pattern): return [bits[i - 1] for i in pattern]
def shift(l, n): return l[n:] + l[:n]
def xor(a, b): return [i ^ j for i, j in zip(a, b)]

S0 = [[1,0,3,2],[3,2,1,0],[0,2,1,3],[3,1,3,2]]
S1 = [[0,1,2,3],[2,0,1,3],[3,0,1,0],[2,1,0,3]]

def sbox(bits, box):
    row = (bits[0]<<1) | bits[3]
    col = (bits[1]<<1) | bits[2]
    val = box[row][col]
    return [(val>>1)&1, val&1]

def fk(bits, key):
    EP = [4,1,2,3,2,3,4,1]
    P4 = [2,4,3,1]
    L, R = bits[:4], bits[4:]
    temp = xor(permute(R, EP), key)
    return xor(L, permute(sbox(temp[:4], S0)+sbox(temp[4:], S1), P4)) + R

def keys(k):
    P10 = [3,5,2,7,4,10,1,9,8,6]
    P8 = [6,3,7,4,8,5,10,9]
    k = permute(k, P10)
    L, R = shift(k[:5], 1), shift(k[5:], 1)
    K1 = permute(L+R, P8)
    L, R = shift(L, 2), shift(R, 2)
    K2 = permute(L+R, P8)
    return K1, K2

def encrypt(pt, k):
    IP, IPi = [2,6,3,1,4,8,5,7], [4,1,3,5,7,2,8,6]
    K1, K2 = keys(k)
    pt = permute(pt, IP)
    pt = fk(pt, K1)
    pt = pt[4:] + pt[:4]
    pt = fk(pt, K2)
    return permute(pt, IPi)

# Example
pt = [1,0,1,0,1,0,1,0]
key = [1,0,1,0,0,0,0,0,1,0]
ct = encrypt(pt, key)
print("Cipher:", ''.join(map(str, ct)))


Cipher: 10001101


In [ ]:
def permute(bits, pattern): return [bits[i - 1] for i in pattern]
def shift(l, n): return l[n:] + l[:n]
def xor(a, b): return [i ^ j for i, j in zip(a, b)]

S0 = [[1,0,3,2],[3,2,1,0],[0,2,1,3],[3,1,3,2]]
S1 = [[0,1,2,3],[2,0,1,3],[3,0,1,0],[2,1,0,3]]

def sbox(bits, box):
    row = (bits[0]<<1) | bits[3]
    col = (bits[1]<<1) | bits[2]
    val = box[row][col]
    return [(val>>1)&1, val&1]

def fk(bits, key, step=''):
    EP = [4,1,2,3,2,3,4,1]
    P4 = [2,4,3,1]
    L, R = bits[:4], bits[4:]
    temp = xor(permute(R, EP), key)
    s_out = sbox(temp[:4], S0) + sbox(temp[4:], S1)
    p4 = permute(s_out, P4)
    result = xor(L, p4) + R
    print(f"{step}fk: L={L}, R={R}, EP+XOR={temp}, Sboxes={s_out}, P4={p4}, Result={result}")
    return result

def keys(k):
    P10 = [3,5,2,7,4,10,1,9,8,6]
    P8 = [6,3,7,4,8,5,10,9]
    k = permute(k, P10)
    L, R = shift(k[:5], 1), shift(k[5:], 1)
    K1 = permute(L+R, P8)
    L, R = shift(L, 2), shift(R, 2)
    K2 = permute(L+R, P8)
    print(f"Key Generation:\n  P10 = {k}, K1 = {K1}, K2 = {K2}")
    return K1, K2

def encrypt(pt, k):
    IP, IPi = [2,6,3,1,4,8,5,7], [4,1,3,5,7,2,8,6]
    print(f"Plaintext = {pt}")
    K1, K2 = keys(k)
    pt = permute(pt, IP)
    print(f"After IP = {pt}")
    pt = fk(pt, K1, 'Round 1 ')
    pt = pt[4:] + pt[:4]
    print(f"After swap = {pt}")
    pt = fk(pt, K2, 'Round 2 ')
    ct = permute(pt, IPi)
    print(f"Cipher = {ct}")
    return ct

# Input
pt = [1,0,1,0,1,0,1,0]             # 8-bit plaintext
key = [1,0,1,0,0,0,0,0,1,0]        # 10-bit key

# Run
cipher = encrypt(pt, key)
print("Final Cipher Text:", ''.join(map(str, cipher)))


Plaintext = [1, 0, 1, 0, 1, 0, 1, 0]
Key Generation:
  P10 = [1, 0, 0, 0, 0, 0, 1, 1, 0, 0], K1 = [1, 0, 1, 0, 0, 1, 0, 0], K2 = [0, 1, 0, 0, 0, 0, 1, 1]
After IP = [0, 0, 1, 1, 0, 0, 1, 1]
Round 1 fk: L=[0, 0, 1, 1], R=[0, 0, 1, 1], EP+XOR=[0, 0, 1, 1, 0, 0, 1, 0], Sboxes=[1, 0, 0, 1], P4=[0, 1, 0, 1], Result=[0, 1, 1, 0, 0, 0, 1, 1]
After swap = [0, 0, 1, 1, 0, 1, 1, 0]
Round 2 fk: L=[0, 0, 1, 1], R=[0, 1, 1, 0], EP+XOR=[0, 1, 1, 1, 1, 1, 1, 1], Sboxes=[0, 0, 1, 1], P4=[0, 1, 1, 0], Result=[0, 1, 0, 1, 0, 1, 1, 0]
Cipher = [1, 0, 0, 0, 1, 1, 0, 1]
Final Cipher Text: 10001101


In [ ]:
def permute(bits, pattern): return [bits[i - 1] for i in pattern]
def shift(l, n): return l[n:] + l[:n]
def xor(a, b): return [i ^ j for i, j in zip(a, b)]

S0 = [[1,0,3,2],[3,2,1,0],[0,2,1,3],[3,1,3,2]]
S1 = [[0,1,2,3],[2,0,1,3],[3,0,1,0],[2,1,0,3]]

def sbox(bits, box):
    row = (bits[0]<<1) | bits[3]
    col = (bits[1]<<1) | bits[2]
    val = box[row][col]
    return [(val>>1)&1, val&1]

def fk(bits, key, step=''):
    EP = [4,1,2,3,2,3,4,1]
    P4 = [2,4,3,1]
    L, R = bits[:4], bits[4:]
    temp = xor(permute(R, EP), key)
    s_out = sbox(temp[:4], S0) + sbox(temp[4:], S1)
    p4 = permute(s_out, P4)
    result = xor(L, p4) + R
    print(f"{step}fk:\n  L = {L}, R = {R}\n  EP(R) ⊕ Key = {temp}\n  S-box output = {s_out}, P4 = {p4}\n  Result = {result}")
    return result

def keys(k):
    P10 = [3,5,2,7,4,10,1,9,8,6]
    P8 = [6,3,7,4,8,5,10,9]
    k = permute(k, P10)
    L, R = shift(k[:5], 1), shift(k[5:], 1)
    K1 = permute(L+R, P8)
    L, R = shift(L, 2), shift(R, 2)
    K2 = permute(L+R, P8)
    print(f"\nKey Generation:\n  P10 = {k}\n  K1  = {K1}\n  K2  = {K2}")
    return K1, K2

def encrypt(pt, k):
    IP, IPi = [2,6,3,1,4,8,5,7], [4,1,3,5,7,2,8,6]
    print(f"\nPlaintext = {pt}")
    K1, K2 = keys(k)
    pt = permute(pt, IP)
    print(f"\nAfter IP  = {pt}")
    pt = fk(pt, K1, 'Round 1 ')
    pt = pt[4:] + pt[:4]
    print(f"\nAfter swap = {pt}")
    pt = fk(pt, K2, 'Round 2 ')
    ct = permute(pt, IPi)
    print(f"\nAfter IP⁻¹ = {ct}")
    return ct

def str_to_bits(s):
    return [int(c) for c in s]

# --- USER INPUT ---
plain_str = input("Enter 8-bit plaintext (e.g. 10101010): ")
key_str = input("Enter 10-bit key (e.g. 1010000010): ")

# Input validation
if len(plain_str) != 8 or len(key_str) != 10 or not set(plain_str+key_str).issubset({'0', '1'}):
    print("Invalid input! Please enter correct bit lengths (8 for plaintext, 10 for key).")
else:
    pt = str_to_bits(plain_str)
    k = str_to_bits(key_str)
    cipher = encrypt(pt, k)
    print("\nFinal Cipher Text:", ''.join(map(str, cipher)))


Enter 8-bit plaintext (e.g. 10101010): 10001010
Enter 10-bit key (e.g. 1010000010): 1000101101

Plaintext = [1, 0, 0, 0, 1, 0, 1, 0]

Key Generation:
  P10 = [0, 1, 0, 1, 0, 1, 1, 0, 1, 0]
  K1  = [1, 1, 0, 0, 1, 0, 1, 0]
  K2  = [1, 0, 0, 1, 1, 0, 0, 1]

After IP  = [0, 0, 0, 1, 0, 0, 1, 1]
Round 1 fk:
  L = [0, 0, 0, 1], R = [0, 0, 1, 1]
  EP(R) ⊕ Key = [0, 1, 0, 1, 1, 1, 0, 0]
  S-box output = [0, 1, 0, 1], P4 = [1, 1, 0, 0]
  Result = [1, 1, 0, 1, 0, 0, 1, 1]

After swap = [0, 0, 1, 1, 1, 1, 0, 1]
Round 2 fk:
  L = [0, 0, 1, 1], R = [1, 1, 0, 1]
  EP(R) ⊕ Key = [0, 1, 1, 1, 0, 0, 1, 0]
  S-box output = [0, 0, 0, 1], P4 = [0, 1, 0, 0]
  Result = [0, 1, 1, 1, 1, 1, 0, 1]

After IP⁻¹ = [1, 0, 1, 1, 0, 1, 1, 1]

Final Cipher Text: 10110111


ICS 2

In [ ]:
# S-AES helper functions and constants
SBOX = [0x9, 0x4, 0xA, 0xB,
        0xD, 0x1, 0x8, 0x5,
        0x6, 0x2, 0x0, 0x3,
        0xC, 0xE, 0xF, 0x7]

SBOX_INV = [0xA, 0x5, 0x9, 0xB,
            0x1, 0x7, 0x8, 0xF,
            0x6, 0x0, 0x2, 0x3,
            0xC, 0x4, 0xD, 0xE]

RCON1 = 0x80
RCON2 = 0x30

def sub_nib(b): return (SBOX[b >> 4] << 4) + SBOX[b & 0x0F]
def sub_nib_inv(b): return (SBOX_INV[b >> 4] << 4) + SBOX_INV[b & 0x0F]

def rot_nib(b): return ((b << 4) | (b >> 4)) & 0xFF

def key_expand(key):
    w = [0]*6
    w[0] = (key >> 8) & 0xFF
    w[1] = key & 0xFF
    w[2] = w[0] ^ RCON1 ^ sub_nib(rot_nib(w[1]))
    w[3] = w[2] ^ w[1]
    w[4] = w[2] ^ RCON2 ^ sub_nib(rot_nib(w[3]))
    w[5] = w[4] ^ w[3]
    return [((w[i] << 8) | w[i+1]) for i in range(0, 6, 2)]

def mult(p1, p2):
    p = 0
    for i in range(4):
        if p2 & 0x1:
            p ^= p1
        hi_bit_set = p1 & 0x8
        p1 = (p1 << 1) & 0xF
        if hi_bit_set:
            p1 ^= 0x3
        p2 >>= 1
    return p

def mix_columns(s):
    return ((mult(s >> 12, 1) << 12) ^ (mult((s >> 8) & 0xF, 4) << 8) ^
            (mult((s >> 4) & 0xF, 4) << 4) ^ (mult(s & 0xF, 1)))

def mix_columns_inv(s):
    return ((mult(s >> 12, 9) << 12) ^ (mult((s >> 8) & 0xF, 2) << 8) ^
            (mult((s >> 4) & 0xF, 2) << 4) ^ (mult(s & 0xF, 9)))

def encrypt(ptext, key):
    keys = key_expand(key)
    state = ptext ^ keys[0]
    state = sub_nib(state >> 8) << 8 | sub_nib(state & 0xFF)
    state = (state & 0xF0F0) >> 4 | (state & 0x0F0F) << 4  # ShiftRows
    state = mix_columns(state)
    state ^= keys[1]
    state = sub_nib(state >> 8) << 8 | sub_nib(state & 0xFF)
    state = (state & 0xF0F0) >> 4 | (state & 0x0F0F) << 4  # ShiftRows
    state ^= keys[2]
    return state

# --- USER INPUT VERSION ---
def get_input():
    pt_hex = input("Enter 16-bit plaintext in hex (e.g., 1234): ")
    key_hex = input("Enter 16-bit key in hex (e.g., abcd): ")
    try:
        pt = int(pt_hex, 16)
        key = int(key_hex, 16)
        if 0 <= pt <= 0xFFFF and 0 <= key <= 0xFFFF:
            ct = encrypt(pt, key)
            print(f"\nEncrypted Ciphertext = {hex(ct)[2:].zfill(4)}")
        else:
            print("Values must be 16-bit (0000 to FFFF).")
    except ValueError:
        print("Invalid input. Please enter valid hexadecimal values.")

get_input()


Enter 16-bit plaintext in hex (e.g., 1234): f0f0
Enter 16-bit key in hex (e.g., abcd): 0f0f

Encrypted Ciphertext = c7e1


Diffie-Hellman Key Exchange

In [ ]:
# Diffie-Hellman Key Exchange

def power(base, exp, mod):
    result = 1
    base = base % mod
    while exp > 0:
        if exp % 2 == 1:
            result = (result * base) % mod
        exp = exp >> 1
        base = (base * base) % mod
    return result

# --- User Inputs ---
p = int(input("Enter a prime number (p): "))           # e.g., 23
g = int(input("Enter a primitive root modulo p (g): ")) # e.g., 5

# Private keys (secret)
a = int(input("Enter Alice's private key: "))          # e.g., 6
b = int(input("Enter Bob's private key: "))            # e.g., 15

# Public keys
A = power(g, a, p)  # Alice sends this
B = power(g, b, p)  # Bob sends this

print(f"\nPublic Key of Alice (A = g^a mod p): {A}")
print(f"Public Key of Bob   (B = g^b mod p): {B}")

# Shared secret key (calculated by both)
s_alice = power(B, a, p)
s_bob = power(A, b, p)

print(f"\nShared Secret Key (computed by Alice): {s_alice}")
print(f"Shared Secret Key (computed by Bob)  : {s_bob}")


Enter a prime number (p): 5
Enter a primitive root modulo p (g): 7
Enter Alice's private key: 12
Enter Bob's private key: 25

Public Key of Alice (A = g^a mod p): 1
Public Key of Bob   (B = g^b mod p): 2

Shared Secret Key (computed by Alice): 1
Shared Secret Key (computed by Bob)  : 1


 Implementation of RSA

In [ ]:
from math import gcd

def modinv(a, m):
    # Extended Euclidean Algorithm
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

def is_prime(n):
    return n > 1 and all(n % i != 0 for i in range(2, int(n**0.5) + 1))

def rsa():
    print("=== RSA Encryption & Decryption ===")

    # --- Inputs ---
    p = int(input("Enter a prime number p: "))   # e.g., 61
    q = int(input("Enter another prime q: "))    # e.g., 53

    if not (is_prime(p) and is_prime(q)):
        print("Both numbers must be prime!")
        return

    n = p * q
    phi = (p - 1) * (q - 1)

    e = int(input(f"Enter public key e (1 < e < {phi}, gcd(e, {phi})=1): "))
    if gcd(e, phi) != 1:
        print("Invalid e. It must be co-prime to φ(n).")
        return

    d = modinv(e, phi)

    print(f"\nPublic Key (e, n): ({e}, {n})")
    print(f"Private Key (d, n): ({d}, {n})")

    # Message input
    msg = int(input("\nEnter message (as a number < n): "))
    if msg >= n:
        print("Message must be less than n.")
        return

    # Encryption
    cipher = pow(msg, e, n)
    print(f"Encrypted Cipher: {cipher}")

    # Decryption
    decrypted = pow(cipher, d, n)
    print(f"Decrypted Message: {decrypted}")

rsa()


=== RSA Encryption & Decryption ===
Enter a prime number p: 61
Enter another prime q: 5
Enter public key e (1 < e < 240, gcd(e, 240)=1): 250
Invalid e. It must be co-prime to φ(n).
